# Set up TVB Simulations from Atlas analysis outputs

In [1]:
# ConWhAt stuff
from conwhat import VolConnAtlas,StreamConnAtlas,VolTractAtlas,StreamTractAtlas
from conwhat.viz.volume import plot_vol_scatter

# Neuroimaging stuff
import nibabel as nib
from nilearn.plotting import (plot_stat_map,plot_surf_roi,plot_roi,
                             plot_connectome,find_xyz_cut_coords)
from nilearn.image import resample_to_img

# Viz stuff
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

# Generic stuff
import glob, numpy as np, pandas as pd, networkx as nx
from datetime import datetime

/global/home/hpc3230/Software/miniconda2/envs/jupyter/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


We now use the synthetic lesion constructed in the previous example in a ConWhAt lesion analysis.

In [2]:
lesion_file = 'synthetic_lesion_20mm_sphere_-46_-60_6.nii.gz' # we created this file from scratch in the previous example

In [3]:
cw_atlases_dir = '/global/scratch/hpc3230/Data/conwhat_atlases'  # change this accordingly
atlas_name = 'CWL2k8Sc33Vol3d100s_v01'
atlas_dir = '%s/%s' %(cw_atlases_dir, atlas_name)

In [4]:
cw_vca = VolConnAtlas(atlas_dir=atlas_dir)

loading file mapping
loading vol bbox
loading connectivity


In [5]:
idxs = 'all' # alternatively, something like: range(1,100), indicates the first 100 cnxns (rows in .vmfs)

In [6]:
jlc_dir = '/global/scratch/hpc3230/joblib_cache_dir' # this is the cache dir where joblib writes temporary files
lo_df,lo_nx = cw_vca.compute_hit_stats(lesion_file,idxs,n_jobs=4,joblib_cache_dir=jlc_dir)

computing hit stats for roi synthetic_lesion_20mm_sphere_-46_-60_6.nii.gz


In [7]:
tpr_adj = nx.to_pandas_adjacency(lo_nx,weight='TPR')
cpr_adj = nx.to_pandas_adjacency(lo_nx,weight='corr_thrbin')

In [8]:
parc_img = cw_vca.region_nii
parc_dat = parc_img.get_data()
parc_vals = np.unique(parc_dat)[1:]

ccs = {roival: find_xyz_cut_coords(nib.Nifti1Image((dat==roival).astype(int),img.affine),
                                   activation_threshold=0) for roival in roivals}
ccs_arr = np.array(ccs.values())

NameError: name 'roivals' is not defined

In [ ]:
import sys
sys.path.append('~/Code/libraries_of_mine/github/tvb-library')
sys.path.append('~/Code/libraries_of_mine/github/tvb-data')

from tvb.simulator.lab import *

In [ ]:
gamma_sp = 1.21
epsilon_sp =  12.3083

# is this the correct? 
# - units in Spiegler are S^-1. 
# - default value for g2do d is 0.02
# - so this gives 0.07674
eta_sp = (1/1000.) * 76.74    #eta_sp = 76.74 # 1. # 76.74 ##1. # 76.74 # 1.



sp_g2do_params = dict(d = eta_sp,
                      tau = 1.,
                      f = 1.,
                      e = 0., 
                      g = -gamma_sp,
                      alpha = 1., 
                      gamma = 1., 
                      c = 0.,
                      b= -epsilon_sp, # should not be negative? LOOKS LIKE IT AL COMES DOWN TO THIS PARAM
                      beta = 0,
                      a = 0.)

In [ ]:
def run_g2do_sim(conn,continue_sim=None, cs=1., D=0.001, cv=3.0, dt=0.5, 
                simlen=1e3,int_type='heun_stoch', burn_in=500,
                 g2do_params={},initconds = None,return_B=True,return_V=True):
    
    if int_type=='heun_stoch':
        solver = integrators.HeunStochastic(dt=dt,noise=noise.Additive(nsig=array([D])))
    elif int_type=='euler_stoch':
        solver = integrators.EulerStochastic(dt=dt,noise=noise.Additive(nsig=array([D])))
    elif int_type == 'heun_det':
        solver = integrators.HeunDeterministic(dt=dt)
        
    if continue_sim == None:
      sim = simulator.Simulator(
          model=models.Generic2dOscillator(**g2do_params),
          connectivity=conn,
          coupling=coupling.Linear(a=cs),
          integrator=solver,
          monitors=(monitors.TemporalAverage(period=5.0),# 200 Hz
                    monitors.Bold())
      )
      sim.configure()
        
    else: 
      sim = continue_sim
    
    if initconds:
        _ = sim.run(simulation_length=5.)
        sim.history = np.ones_like(sim.history)*initconds
            
    if burn_in:
        _ = sim.run(simulation_length=burn_in)
        
    res = sim.run(simulation_length=simlen)
    (V_ts,V_ys),(B_ts,B_ys) = res
    
    if return_B:
      df_B = pd.DataFrame(np.squeeze(B_ys), index=B_ts)        
    else: 
      df_B = None
    
    if return_V:
      df_V = pd.DataFrame(np.squeeze(V_ys), index=V_ts)
    else: 
     df_V = None
        
    return df_V,df_B,sim

In [ ]:
Build a connectivity 

In [ ]:
conn = connectivity.Connectivity()
conn.weights = cw_vca.weights
conn.

In [ ]:
origconn = connectivity.Connectivity(load_default=True)
origconn.configure()
res = run_g2do_sim(origconn,initconds=1.,int_type='heun_det')#,burn_in=None)
tvbcaV_ic1hd_ref,tvbcaB_ic1hd_ref,tvbcasims_ic1hd_ref = res    
tvbcaVc_ic1hd_ref = tvbcaV_ic1hd_ref.corr()
res = run_g2do_sim(origconn,initconds=None,int_type='heun_det')#,burn_in=None)
tvbcaV_icrhd_ref,tvbcaB_icrhd_ref,tvbcasims_icrhd_ref = res    
tvbcaVc_icrhd_ref = tvbcaV_icrhd_ref.corr()
res = run_g2do_sim(origconn,initconds=None,int_type='heun_stoch')#,burn_in=None)
tvbcaV_icrhs_ref,tvbcaB_icrhs_ref,tvbcasims_icrhs_ref = res    
tvbcaVc_icrhs_ref = tvbcaV_icrhs_ref.corr()